In [233]:
import pandas as pd
import collections
from sklearn.ensemble import RandomForestClassifier

from collections import defaultdict
from pymongo import MongoClient
import re
import numpy as np
from gensim.models import Word2Vec
from datetime import datetime
from nltk.tokenize import word_tokenize
import gensim

In [ ]:
train_data=pd.read_json("train_original_tweets_top1000.json")

In [140]:
test_data=pd.read_json("test_original_tweets_top1000.json")

In [142]:
train_data.head()

,created_at,entities,retweet_count,retweet_median,retweeted,screen_name,text,time
0,2018-04-09 18:04:33,"{'symbols': [], 'hashtags': [], 'user_mentions...",1230,899.0,False,katyperry,I can't wait for you to 👁witness👁 my baby bird...,1523297073000
1,2018-04-09 02:57:37,"{'symbols': [], 'hashtags': [{'text': 'america...",1478,899.0,False,katyperry,Who's ready for #americanidol ⁉️🙋🏼Live on Inst...,1523242657000
10,2018-04-03 04:27:30,"{'symbols': [], 'hashtags': [{'text': 'america...",411,899.0,False,katyperry,👂🏻@calebleemusic WE ARE LISTENING 👂🏻AND WE LIK...,1522729650000
100,2018-03-23 17:52:04,"{'symbols': [], 'hashtags': [], 'user_mentions...",1016,899.0,False,katyperry,🚨❗THIS JUST IN❗🚨\nNow your little one can make...,1521827524000
1000,2018-01-29 20:06:42,"{'symbols': [], 'hashtags': [], 'user_mentions...",7339,6050.5,False,ladygaga,Here’s the Piano Version of “Joanne” music vid...,1517256402000


In [143]:
len(train_data)

295720

In [144]:
test_data.head()

,created_at,entities,retweet_count,retweet_median,retweeted,screen_name,text,time
0,2018-04-16 23:48:44,"{'symbols': [], 'hashtags': [{'text': 'America...",1320,899.0,False,katyperry,I have good news and bad news today - ☹️: I wo...,1523922524000
1,2018-04-15 23:38:20,"{'symbols': [], 'hashtags': [{'text': 'america...",1559,899.0,False,katyperry,"Ooh, almost time for another #americanidol! 👏🏼...",1523835500000
10,2018-04-10 18:49:07,"{'symbols': [], 'hashtags': [], 'user_mentions...",5770,43399.0,True,justinbieber,RT @poobear: Thanks for this @HannahStocking! ...,1523386147000
100,2018-04-16 14:04:03,"{'symbols': [], 'hashtags': [], 'user_mentions...",81,111.0,False,YouTube,Monday mood. https://t.co/IFNJryvUcE,1523887443000
1000,2018-04-12 23:17:04,"{'symbols': [], 'hashtags': [], 'user_mentions...",96,104.0,False,nytimes,For a nuanced picture of Lebanon and its peopl...,1523575024000


In [154]:
len(test_data)

80218

In [228]:
def get_average_word_length(words,english_punctuations):
    word_count=0
    word_total_length=0
    for word in words:
        if word not in english_punctuations:
            word_total_length+=len(word)
            word_count+=1
    return word_total_length/word_count

In [234]:
def generate_common_features(train_df):
    english_punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']
    train_df["tokenized_words"]=train_df["text"].apply(lambda x:word_tokenize(x))
    train_df["hashtag_num"]=train_df["entities"].apply(lambda x:len(x["hashtags"]))
    train_df["link_num"]=train_df["entities"].apply(lambda x:len(x["urls"]))
    train_df["mention_num"]=train_df["entities"].apply(lambda x:len(x["user_mentions"]))
    train_df["time_of_date"]=train_df["created_at"].apply(lambda x:x.hour*24+x.minute)
    train_df["len_of_tweet"]=train_df["text"].apply(lambda x:len(x))
    train_df["avg_len_of_word"]=train_df["tokenized_words"].apply(lambda x:get_average_word_length(x,english_punctuations))


In [146]:
name_dict={}
temp_count=0
for i in train_data["screen_name"]:
    if i in name_dict:
        continue
    else:
        name_dict[i]=temp_count
        temp_count+=1

for i in test_data["screen_name"]:
    if i in name_dict:
        continue
    else:
        name_dict[i]=temp_count
        temp_count+=1
        
train_data["screen_name"]=[name_dict[i] for i in train_data["screen_name"]]
test_data["screen_name"]=[name_dict[i] for i in test_data["screen_name"]]


In [147]:
len(train_data)

295720

In [107]:
for i in test_data["entities"].head():
    print(i)
    break

{'symbols': [], 'hashtags': [{'text': 'AmericanIdol', 'indices': [73, 86]}], 'user_mentions': [], 'urls': [{'display_url': 'twitter.com/i/web/status/9…', 'expanded_url': 'https://twitter.com/i/web/status/986028650607734784', 'indices': [117, 140], 'url': 'https://t.co/VSMnfhh1EG'}]}


In [148]:
test_data["hashtags_num"]=[len(i['hashtags']) for i in test_data["entities"]]

In [149]:
train_data["hashtags_num"]=[len(i['hashtags']) for i in train_data["entities"]]

In [150]:
train_data["user_mentions"]=[len(i['user_mentions']) for i in train_data["entities"]]
test_data["user_mentions"]=[len(i['user_mentions']) for i in test_data["entities"]]
train_data["urls"]=[len(i['urls']) for i in train_data["entities"]]
test_data["urls"]=[len(i['urls']) for i in test_data["entities"]]

In [151]:
train_data["hashtags_num"]=[len(i['hashtags']) for i in train_data["entities"]]
test_data["hashtags_num"]=[len(i['hashtags']) for i in test_data["entities"]]

ValueError: labels ['entities'] not contained in axis

In [155]:
#4 features involving most tweetable words
most_tweetable_words={"you":0,"twitter":1,"please":2,"retweet":3,"post":4,"blog":5,"social":6,
                     "free":7,"media":8,"help":9,"great":10,"10":11,"follow":12,"how":13,
                      "to":14,"top":15,"check":16,"out":17,"this":18,"people":19,"votr":20,
                     "?":21}
def add_most_tweetable_words_features(pd):
    pd["aggreg_exist"]=[1 if any(t in most_tweetable_words for t in i.split()) else 0 for i in pd["text"]]
    pd["aggreg_tf"]=[sum([1 if t in most_tweetable_words else 0 for t in i.split()]) for i in pd["text"]]
    for i in most_tweetable_words:
        pd[i+"_exist"]=[1 if any(t==i for t in j.split()) else 0 for j in pd["text"]]
        pd[i+"_tf"]=[sum([1 if t==i else 0 for t in j.split()]) for j in pd["text"]]

def add_question_features(pd):
    pd["ques_exist"]=[1 if collections.Counter(i)["?"]!=0 else 0 for i in pd["text"]]
    pd["ques_exist"]=[collections.Counter(i)["?"] for i in pd["text"]]
    

In [235]:
generate_common_features(train_data)
generate_common_features(test_data)

KeyError: 'entities'

In [ ]:
test_data=test_data.drop(["entities"],axis=1)
train_data=train_data.drop(["entities"],axis=1)
print(len(test_data))

In [156]:
add_most_tweetable_words_features(train_data)
add_question_features(train_data)
print(train_data.head())

              created_at  retweet_count  retweet_median  retweeted  \
0    2018-04-09 18:04:33           1230           899.0      False   
1    2018-04-09 02:57:37           1478           899.0      False   
10   2018-04-03 04:27:30            411           899.0      False   
100  2018-03-23 17:52:04           1016           899.0      False   
1000 2018-01-29 20:06:42           7339          6050.5      False   

      screen_name                                               text  \
0               0  I can't wait for you to 👁witness👁 my baby bird...   
1               0  Who's ready for #americanidol ⁉️🙋🏼Live on Inst...   
10              0  👂🏻@calebleemusic WE ARE LISTENING 👂🏻AND WE LIK...   
100             0  🚨❗THIS JUST IN❗🚨\nNow your little one can make...   
1000            1  Here’s the Piano Version of “Joanne” music vid...   

               time  hashtags_num  user_mentions  urls     ...      people_tf  \
0     1523297073000             0              1     1     ...   

In [157]:
len(train_data)

295720

In [158]:
add_most_tweetable_words_features(test_data)
add_question_features(test_data)
print(test_data.head())

              created_at  retweet_count  retweet_median  retweeted  \
0    2018-04-16 23:48:44           1320           899.0      False   
1    2018-04-15 23:38:20           1559           899.0      False   
10   2018-04-10 18:49:07           5770         43399.0       True   
100  2018-04-16 14:04:03             81           111.0      False   
1000 2018-04-12 23:17:04             96           104.0      False   

      screen_name                                               text  \
0               0  I have good news and bad news today - ☹️: I wo...   
1               0  Ooh, almost time for another #americanidol! 👏🏼...   
10            732  RT @poobear: Thanks for this @HannahStocking! ...   
100           228               Monday mood. https://t.co/IFNJryvUcE   
1000          795  For a nuanced picture of Lebanon and its peopl...   

               time  hashtags_num  user_mentions  urls     ...      people_tf  \
0     1523922524000             1              0     1     ...   

In [201]:
# train_target=train_data["retweet_median"]
train_target["diff"]=train_data["retweet_count"]-train_data["retweet_median"]
train_target["label"]=[1 if i>=0 else 0 for i in train_target["diff"]]

test_target["diff"]=test_data["retweet_count"]-test_data["retweet_median"]
test_target["label"]=[1 if i>=0 else 0 for i in test_target["diff"]]

In [202]:
train_features=train_data.drop(["text"],axis=1)
test_features=test_data.drop(["text"],axis=1)

In [203]:
train_features=train_features.drop(["created_at"],axis=1)
test_features=test_features.drop(["created_at"],axis=1)

In [204]:
test_features=test_features.drop(["retweet_median"],axis=1)
test_features=test_features.drop(["retweet_count"],axis=1)

In [205]:
train_features=train_features.drop(["retweet_median"],axis=1)
train_features=train_features.drop(["retweet_count"],axis=1)

In [213]:
print(train_features.head())
print(len(train_target))
print(len(test_features))
print(len(test_target))

      retweeted  screen_name           time  hashtags_num  user_mentions  \
0         False            0  1523297073000             0              1   
1         False            0  1523242657000             1              0   
10        False            0  1522729650000             1              1   
100       False            0  1521827524000             0              1   
1000      False            1  1517256402000             0              0   

      urls  aggreg_exist  aggreg_tf  ?_exist  ?_tf     ...      people_tf  \
0        1             1          2        0     0     ...              0   
1        1             0          0        0     0     ...              0   
10       0             0          0        0     0     ...              0   
100      1             1          1        0     0     ...              0   
1000     1             1          1        0     0     ...              0   

      retweet_exist  retweet_tf  10_exist  10_tf  you_exist  you_tf  \
0        

In [211]:
print(sum([1 if i==1 else 0 for i in train_target["label"]]))

160314


In [226]:
#training
import sklearn
clf = sklearn.ensemble.AdaBoostClassifier()  
RandomForestModel= clf.fit(train_features,train_target["label"])
r = clf.score(test_features , test_target["label"])  
print(r)

0.5584282829290185


In [227]:
r2 = clf.score(train_features , train_target["label"])  
print(r2)

0.5759738942242661
